quick code to add centroid info to codex cells csv, from segmentation files.

Centroide info was not used in MaxFuse but used in downstream analysis eg GC analysis or visualization

In [2]:
import numpy as np
import pandas as pd

In [3]:
formatch = pd.read_csv("formatch_clusters_x28_y715V2.csv")
formatch

,Unnamed: 0,X,cellLabelInImage,cellSize,HOECHST1,CD38,CD19,CD31,Vimentin,CD22,...,CD40,CD274,CD27,CD104,CD273,FAPalpha,Ecadherin,PointNum,cluster.term,seurat_res1.0
0,6118,8142,1,51,6267.294118,0.143310,0.614983,0.016350,0.299605,1.000000,...,0.347517,0.236198,0.187309,0.025635,0.026118,0.214350,0.178581,reg001_X02_Y07_Z09,B-CD22-CD40,8
1,6119,8143,2,53,6805.830189,0.069681,1.000000,0.153656,1.000000,0.718178,...,0.524732,0.430924,0.188372,0.073011,1.000000,0.338075,0.342726,reg001_X02_Y07_Z09,B-CD22-CD40,7
2,6120,8144,3,57,4889.789474,0.438127,1.000000,0.044091,0.273363,0.443502,...,0.265005,0.308467,0.046056,0.286851,1.000000,0.789193,0.340269,reg001_X02_Y07_Z09,B-Ki67,6
3,6122,8147,6,56,4450.660714,0.000000,1.000000,0.050185,0.274332,1.000000,...,0.449587,0.000000,0.077036,0.028937,0.000000,0.166796,0.082396,reg001_X02_Y07_Z09,B-CD22-CD40,0
4,6123,8148,7,79,8951.000000,0.114900,0.412287,0.039873,0.418782,0.769603,...,0.137616,0.222925,0.119464,0.018779,0.000000,0.067358,0.166426,reg001_X02_Y07_Z09,B-CD22-CD40,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178914,244723,307664,4081,46,4188.065000,0.000000,0.516711,0.079410,0.000000,0.904165,...,1.000000,0.000000,0.000000,0.010496,0.000000,0.000000,0.000000,reg001_X08_Y15_Z06,B-CD22-CD40,0
178915,244724,307665,4082,65,4169.923000,0.210978,0.034039,0.010116,0.024256,0.000000,...,0.000000,0.074103,1.000000,0.032785,0.104098,0.257564,0.106543,reg001_X08_Y15_Z06,CD4 T,1
178916,244725,307666,4083,75,9355.747000,0.086992,0.020658,0.005740,0.334716,0.000000,...,0.000000,0.578852,0.462894,0.077727,0.280892,0.638970,0.230461,reg001_X08_Y15_Z06,CD4 T,1
178917,244726,307667,4084,77,7284.675000,0.143877,0.785678,0.138733,0.184268,1.000000,...,0.974257,0.692838,0.083775,0.079336,0.120146,0.439836,0.316185,reg001_X08_Y15_Z06,B-CD22-CD40,0


In [4]:
## some code from shuxiao chen github
import os
import re

import numpy as np
import pandas as pd
import scipy
from skimage.measure import regionprops
from scipy.io import loadmat
from skimage.io import imread
from skimage.transform import resize

def get_cell_idx_partition(df):
    """
    Return partition s.t. partition[i] = [start_i, end_i] are the
    starting and ending indices (closed interval) in df for the i-th FOV (indexing from 1).
    """
    starts = [0]
    for i, curr_cell_idx in enumerate(df['cellLabelInImage']):
        if i > 0 and curr_cell_idx < df['cellLabelInImage'].iloc[i - 1]:
            starts.append(i)
    partition = []
    for i, s in enumerate(starts):
        if i < len(starts) - 1:
            partition.append([s, starts[i + 1] - 1])
        else:
            partition.append([s, df.shape[0] - 1])
    return partition

def add_cell_locations(
        df, path_to_segmentation, shape_of_each_view=(1008, 1344), verbose=True
):
    """
    Add three new columns to df: location coordinates (x, y) as well as which FOV is each cell in.
    """
    # get the splitting points of different FOVs
    partition = iter(get_cell_idx_partition(df))
    centroid_x = []
    centroid_y = []
    all_folders = os.listdir(path_to_segmentation)
    for x in range(2, 9):
        for y in range(7, 16):
            topleft = [(y-7) * shape_of_each_view[0], (x-2) * shape_of_each_view[1]]
            if verbose:
                print("Now at ({}, {}), top-left coordinate is {}".format(x, y, topleft), flush=True)

            # get folder name
            x_str, y_str = ('0' + str(x))[-2:], ('0' + str(y))[-2:]
            curr_folder = [name for name in all_folders if re.search('X'+x_str+'_Y'+y_str, name)]
            assert len(curr_folder) == 1
            curr_folder = curr_folder[0]

            seg = scipy.io.loadmat(
                path_to_segmentation +
                curr_folder + '/H3_CD45_Vim_HLA_Mesmer0111Whole_AutoHist_mpp0.6/segmentationParams.mat'
            )['newLmod']
            # get unique labels, excluding zero
            unique_seg_labels = list(np.unique(seg.flatten()))[1:]
            # calculate centroids
            props = regionprops(seg)
            # unique labels should align with props
            assert len(unique_seg_labels) == len(props)
            # build dict of seg_label: x and seg_label: y
            seg_label_to_x = {}
            seg_label_to_y = {}
            for i in range(len(props)):
                seg_label_to_x[unique_seg_labels[i]] = props[i]['centroid'][0] + topleft[0]
                seg_label_to_y[unique_seg_labels[i]] = props[i]['centroid'][1] + topleft[1]
            # fill the centroids of this segment of df
            start, end = next(partition)
            for i in range(start, end + 1):
                centroid_x.append(seg_label_to_x[df.iloc[i]['cellLabelInImage']])
                centroid_y.append(seg_label_to_y[df.iloc[i]['cellLabelInImage']])
    # add new columns
    df['centroid_x'] = centroid_x
    df['centroid_y'] = centroid_y

In [5]:
# add centroid location from segmentation masks
add_cell_locations(formatch,'/home/bkzhu/super_mario/tonsil_v2/Codex/Images_singleChannel_0503seg/')

Now at (2, 7), top-left coordinate is [0, 0]
Now at (2, 8), top-left coordinate is [1008, 0]
Now at (2, 9), top-left coordinate is [2016, 0]
Now at (2, 10), top-left coordinate is [3024, 0]
Now at (2, 11), top-left coordinate is [4032, 0]
Now at (2, 12), top-left coordinate is [5040, 0]
Now at (2, 13), top-left coordinate is [6048, 0]
Now at (2, 14), top-left coordinate is [7056, 0]
Now at (2, 15), top-left coordinate is [8064, 0]
Now at (3, 7), top-left coordinate is [0, 1344]
Now at (3, 8), top-left coordinate is [1008, 1344]
Now at (3, 9), top-left coordinate is [2016, 1344]
Now at (3, 10), top-left coordinate is [3024, 1344]
Now at (3, 11), top-left coordinate is [4032, 1344]
Now at (3, 12), top-left coordinate is [5040, 1344]
Now at (3, 13), top-left coordinate is [6048, 1344]
Now at (3, 14), top-left coordinate is [7056, 1344]
Now at (3, 15), top-left coordinate is [8064, 1344]
Now at (4, 7), top-left coordinate is [0, 2688]
Now at (4, 8), top-left coordinate is [1008, 2688]
Now 

In [6]:
formatch

,Unnamed: 0,X,cellLabelInImage,cellSize,HOECHST1,CD38,CD19,CD31,Vimentin,CD22,...,CD27,CD104,CD273,FAPalpha,Ecadherin,PointNum,cluster.term,seurat_res1.0,centroid_x,centroid_y
0,6118,8142,1,51,6267.294118,0.143310,0.614983,0.016350,0.299605,1.000000,...,0.187309,0.025635,0.026118,0.214350,0.178581,reg001_X02_Y07_Z09,B-CD22-CD40,8,1.725490,510.843137
1,6119,8143,2,53,6805.830189,0.069681,1.000000,0.153656,1.000000,0.718178,...,0.188372,0.073011,1.000000,0.338075,0.342726,reg001_X02_Y07_Z09,B-CD22-CD40,7,1.754717,881.490566
2,6120,8144,3,57,4889.789474,0.438127,1.000000,0.044091,0.273363,0.443502,...,0.046056,0.286851,1.000000,0.789193,0.340269,reg001_X02_Y07_Z09,B-Ki67,6,1.964912,1287.526316
3,6122,8147,6,56,4450.660714,0.000000,1.000000,0.050185,0.274332,1.000000,...,0.077036,0.028937,0.000000,0.166796,0.082396,reg001_X02_Y07_Z09,B-CD22-CD40,0,1.910714,660.875000
4,6123,8148,7,79,8951.000000,0.114900,0.412287,0.039873,0.418782,0.769603,...,0.119464,0.018779,0.000000,0.067358,0.166426,reg001_X02_Y07_Z09,B-CD22-CD40,4,2.291139,851.860759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178914,244723,307664,4081,46,4188.065000,0.000000,0.516711,0.079410,0.000000,0.904165,...,0.000000,0.010496,0.000000,0.000000,0.000000,reg001_X08_Y15_Z06,B-CD22-CD40,0,9069.152174,8730.326087
178915,244724,307665,4082,65,4169.923000,0.210978,0.034039,0.010116,0.024256,0.000000,...,1.000000,0.032785,0.104098,0.257564,0.106543,reg001_X08_Y15_Z06,CD4 T,1,9069.292308,8789.384615
178916,244725,307666,4083,75,9355.747000,0.086992,0.020658,0.005740,0.334716,0.000000,...,0.462894,0.077727,0.280892,0.638970,0.230461,reg001_X08_Y15_Z06,CD4 T,1,9068.853333,9216.053333
178917,244726,307667,4084,77,7284.675000,0.143877,0.785678,0.138733,0.184268,1.000000,...,0.083775,0.079336,0.120146,0.439836,0.316185,reg001_X08_Y15_Z06,B-CD22-CD40,0,9068.948052,8273.857143
